# Bailey's Smoke Test

Using local machine with RTX 3080, CUDA 12.8, and python 3.12.7

Installed pytorch using `pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126`

In [10]:
import torch


def is_cuda_available():
    return torch.cuda.is_available()


if __name__ == "__main__":
    if is_cuda_available():
        print("CUDA is available!")
        print(f"CUDA Device Count: {torch.cuda.device_count()}")
        print(f"Current CUDA Device: {torch.cuda.current_device()}")
        print(f"Device Name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
    else:
        print("CUDA is not available.")

CUDA is available!
CUDA Device Count: 1
Current CUDA Device: 0
Device Name: NVIDIA GeForce RTX 3080


In [11]:
import torch
from torch import nn
import torch.nn.functional as F
import lightning as L
import torchmetrics


class BaselineModel(L.LightningModule):
    def __init__(self, num_classes=10):
        super().__init__()

        self.estimator = nn.Sequential(
            nn.Linear(64 * 64, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
        )

        self.accuracy = torchmetrics.Accuracy(
            task="multiclass", num_classes=num_classes
        )

    def forward(self, x):
        x = x.view(x.shape[0], -1)

        return self.estimator(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)

        self.log("val_accuracy", self.accuracy)
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)

        self.log("test_accuracy", self.accuracy)
        self.log("test_loss", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

The Imagenette dataset is a smaller subset of 10 easily classified classes from Imagenet. It is available to download from `torchvision`, as shown in the cell below. There are 3 different sizes of the images available. Feel free to use whichever version you prefer. It might make a difference in the performance of your model.

**Note: After downloading the Imagenette dataset, you will need to set `download=False` in the cell below to avoid errors.**

In [12]:
from torchvision import transforms
from torchvision.datasets import Imagenette
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint


# Prepare the dataset
train_transforms = transforms.Compose(
    [
        transforms.CenterCrop(160),
        transforms.Resize(64),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
        transforms.Grayscale(),
    ]
)

test_transforms = transforms.Compose(
    [
        transforms.CenterCrop(160),
        transforms.Resize(64),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
        transforms.Grayscale(),
    ]
)

train_dataset = Imagenette(
    "data/imagenette/train/",
    split="train",
    size="160px",
    download=False,
    transform=train_transforms,
)


# Use 10% of the training set for validation
train_set_size = int(len(train_dataset) * 0.9)
val_set_size = len(train_dataset) - train_set_size

seed = torch.Generator().manual_seed(42)
train_dataset, val_dataset = torch.utils.data.random_split(
    train_dataset, [train_set_size, val_set_size], generator=seed
)
val_dataset.dataset.transform = test_transforms

# Use DataLoader to load the dataset
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=128, num_workers=8, shuffle=True, persistent_workers=True
)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=128, num_workers=8, shuffle=False, persistent_workers=True
)


# Configure the test dataset
test_dataset = Imagenette(
    "data/imagenette/test/",
    split="val",
    size="160px",
    download=False,
    transform=test_transforms,
)

model = BaselineModel()

# Add EarlyStopping
early_stop_callback = EarlyStopping(monitor="val_loss", mode="min", patience=5)

# Configure Checkpoints
checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min")

In [13]:
# Fit the model
trainer = L.Trainer(
    callbacks=[early_stop_callback, checkpoint_callback], max_epochs=1000
)
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | estimator | Sequential         | 4.8 M  | train
1 | accuracy  | MulticlassAccuracy | 0      | train
---------------------------------------------------------
4.8 M     Trainable params
0         Non-trainable params
4.8 M     Total params
19.148    Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Epoch 8: 100%|██████████| 67/67 [00:03<00:00, 21.27it/s, v_num=3]           


In [14]:
# Evaluate the model on the test set
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=256, num_workers=8, shuffle=False
)
trainer.test(model=model, dataloaders=test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\baile\Documents\GitHub\cse6363-assignment3\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing DataLoader 0: 100%|██████████| 16/16 [00:02<00:00,  6.65it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.3431847095489502
        test_loss           2.4833033084869385
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_accuracy': 0.3431847095489502, 'test_loss': 2.4833033084869385}]